<a href="https://colab.research.google.com/gist/22961-Deep-learning/ba52a5fe649a2eb272196d9b584dc2b1/22961_2_2_3_sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
trans=torchvision.transforms.Compose(
    [torchvision.transforms.PILToTensor(),
     torchvision.transforms.ConvertImageDtype(torch.float)])
train_data_transformed = torchvision.datasets.FashionMNIST(
    root="/22961", train=True, download=True,
    transform=trans)

In [2]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    train_data_transformed, batch_size=60000)
imgs, labels = next(iter(train_dataloader))

In [3]:
print(imgs.size())
imgs=imgs.flatten(start_dim=1)
y_true=torch.nn.functional.one_hot(labels,num_classes=10)
print(imgs.size())
print(y_true.size())

torch.Size([60000, 1, 28, 28])
torch.Size([60000, 784])
torch.Size([60000, 10])


In [4]:
from torch import nn
z=nn.Linear(784,10)
y=nn.Softmax(dim=1)
print(z)

Linear(in_features=784, out_features=10, bias=True)


#True Gradient

In [5]:
%%timeit
y_model=y(z(imgs))
H = -torch.sum(y_true*torch.log(y_model),dim=1)
CE_loss = torch.mean(H)
CE_loss.backward()

97.5 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#Stochastic Gradient

In [6]:
train_dataloader_mini = DataLoader(
    train_data_transformed, batch_size=16, shuffle=True)
minibatch_imgs, minibatch_labels = next(iter(train_dataloader_mini))
minibatch_imgs = minibatch_imgs.flatten(start_dim=1)
minibatch_y_true = torch.nn.functional.one_hot(minibatch_labels,num_classes=10)

In [7]:
%%timeit
minibatch_y_model=y(z(minibatch_imgs))
H = -torch.sum(minibatch_y_true*torch.log(minibatch_y_model),dim=1)
CE_loss = torch.mean(H)
CE_loss.backward()

227 µs ± 5.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# SGD Iteration

In [8]:
alpha=0.1
z.zero_grad()
minibatch_y_model=y(z(minibatch_imgs))
H = -torch.sum(minibatch_y_true*torch.log(minibatch_y_model),dim=1)
CE_loss = torch.mean(H)
CE_loss.backward()
with torch.no_grad():
  z.weight -= alpha*z.weight.grad
  z.bias   -= alpha*z.bias.grad 